# Simulation of the Process with Input Variations

Import PyRolL modules.

In [12]:
import pyroll.basic as pr
import pyroll.export as pre

Import the process data.

In [2]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE, TEMPERATURE, DIAMETER, TEMPERATURE_STD, DIAMETER_STD, create_in_profile
from weiner_variation.sim.config import FIELDS, SAMPLE_COUNT, SEED
from weiner_variation.sim.data_structures import DrawInput

Import further libraries.

In [3]:
import pandas as pd
from copy import deepcopy
from scipy.stats import norm
import numpy as np
import tqdm
from multiprocessing import Pool

Parameters for notebook using papermill.

In [4]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_input_results.csv"

Create distribution functions for input values.

In [5]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

Draw random inputs from distributions.

In [6]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

Define a worker function creating the in profile, running the solution procedure and extracting results.

In [13]:
def worker(draw: DrawInput):    
    ip = create_in_profile(draw.diameter)
    ip.temperature = draw.temperature

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | pre.to_dict(sequence)

Run the simulations using a process pool.

In [14]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|██████████| 10/10 [00:03<00:00,  3.16it/s]


Create a dataframe from results.

In [15]:
df = results |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

,"(draw, diameter)","(draw, temperature)",type,label,in_profile,out_profile,power,log_elongation,max_iteration_count,duration,elongation,iteration_precision,units
0,0.050390,1433.393000,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.312527e+06,3.663753,100,63.767639,39.007456,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
1,0.050772,1413.231456,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.512734e+06,3.678836,100,63.768880,39.600262,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
2,0.051325,1438.005183,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.77...",8.332473e+06,3.700487,100,63.770647,40.467007,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
3,0.049996,1417.283792,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.425830e+06,3.648029,100,63.766337,38.398916,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
4,0.050787,1412.266876,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.522281e+06,3.679425,100,63.768928,39.623611,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
5,0.051227,1425.030163,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.77...",8.438534e+06,3.696674,100,63.770337,40.313000,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
6,0.050200,1433.518216,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.299278e+06,3.656182,100,63.767013,38.713257,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
7,0.050152,1422.000784,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.395029e+06,3.654246,100,63.766853,38.638371,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
8,0.048080,1422.537161,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.75...",8.258330e+06,3.569881,100,63.759813,35.512362,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."
9,0.049793,1416.863055,PassSequence,,"{'type': 'PassSequence.InProfile', 't': 0, 'cr...","{'type': 'PassSequence.OutProfile', 't': 63.76...",8.416118e+06,3.639880,100,63.765660,38.087247,0.001,"[{'type': 'RollPass', 'label': 'R1', 'in_profi..."


Save dataframe to CSV file.

In [9]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)